In [ ]:
from qat.lang.AQASM import Program, H, CNOT, AbstractGate
import numpy as np

In [ ]:
import sys
sys.version

In [ ]:
from helpers import U_ZZ, U_XX, U_YY, U_IZ, U_ZI, U_II

In [ ]:
prog = Program()

n = 2
t = 8

q = prog.qalloc(n+t)

prog.apply(U_ZZ(4.5).ctrl(), [q[0]]+q[t:])
prog.apply(U_XX(2.85).ctrl(), [q[1]]+q[t:])
prog.apply(U_YY(78.8).ctrl(), [q[2]]+q[t:])
prog.apply(U_IZ(78.8).ctrl(), [q[3]]+q[t:])
prog.apply(U_ZI(78.8).ctrl(), [q[4]]+q[t:])
prog.apply(U_II(78.8).ctrl(), [q[5]]+q[t:])


In [ ]:
circ = prog.to_circ(inline=True)

%qatdisplay --svg circ

In [ ]:
import json

with open('hamiltonian_data.json','r') as f:
    l = json.load(f)

In [ ]:
print(l[0].keys())

In [ ]:
from helpers import pea

In [ ]:
t = 1

prog = Program()

qbits = prog.qalloc(t+2) 

prog.apply(pea(l[1],10,t,-0.3), qbits)

circ = prog.to_circ()

%qatdisplay --svg circ 

In [ ]:
from qat.qpus import get_default_qpu
qpu = get_default_qpu()

In [ ]:
job = circ.to_job(qubits = range(t))

In [ ]:
result = qpu.submit(job)

for sample in result:
    print(sample.probability,sample.state)

In [ ]:
import matplotlib.pylab as plt
energy = []
Rs = []

t = 8
p = 1

for coeffs in l:
    Rs.append(coeffs["R"])

    prog = Program()

    qbits = prog.qalloc(t+2) 

    prog.apply(pea(coeffs,p,t,0.3), qbits)

    circ = prog.to_circ()
    job = circ.to_job(qubits = range(t))

    result = qpu.submit(job)

    prob = -1
    

    for sample in result:
        if sample.probability > prob:
            state = sample.state
            prob = sample.probability
            sam = sample
    
    print(state)
    energy.append(sam._state)
    


In [ ]:
plt.plot(Rs, energy)
plt.ylim([0,1000])
    

In [ ]:
U_ZZ = AbstractGate("U_ZZ", [str], arity=2)

In [ ]:
prog = Program()

qbits = prog.qalloc(2)

prog.apply(U_ZZ("dt"), qbits)

circ = prog.to_circ()

%qatdisplay  circ

In [ ]:
from qat.lang.AQASM import RZ
prog = Program()

qbits = prog.qalloc(2)

prog.apply(CNOT, qbits)
prog.apply(RZ(1.5), qbits[1])
prog.apply(CNOT, qbits)

circ = prog.to_circ()

%qatdisplay circ

In [ ]:
ZZ = np.diag([1,-1,-1,1])
XX = np.diag([1,1,1,1])
XX = np.fliplr(XX)
YY = np.diag([-1,1,1,-1])
YY = np.fliplr(YY)

CNOT = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])
I = np.eye(2)
Z = np.diag([1,-1])
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
S = np.diag([1,1j])

print("ZZ",linalg.expm(-1j*ZZ)==np.dot(np.dot(CNOT,np.kron(I,linalg.expm(-1j*Z))),CNOT))
print("---")
print("XX",linalg.expm(-1j*XX)==np.dot(np.dot(CNOT,np.kron(linalg.expm(-1j*X),I)),CNOT)  )
print("---")
A = np.dot(np.kron(I,S), CNOT)
B = np.dot(np.kron(linalg.expm(-1j*Y),I),CNOT)
C = np.kron(I,S.conj())
print("YY",linalg.expm(-1j*YY)==np.dot(A,np.dot(B,C)))